In [4]:
import numpy as np

# ============================================================
# 0. INPUT CƠ BẢN – THAM SỐ THIẾT KẾ
# ============================================================

# Nước và trọng lực
RHO_WATER = 1000.0      # kg/m^3 – mật độ nước
G          = 9.81       # m/s^2 – gia tốc trọng trường

# Vật liệu bê tông
UNIT_WEIGHT_CONCRETE = 24000.0   # N/m^3 ≈ 24 kN/m^3

# Hình học cơ bản của đập (FIXED INPUT)
D  = 50.0     # m – chiều cao cột nước = chiều cao đập theo phương đứng
L  = 50.0     # m – chiều dài đập (chiều ra màn hình trong hình vẽ)
TOP_WIDTH = 2.0   # m – chiều rộng đỉnh (crest width)

# Yêu cầu an toàn
TARGET_FACTOR_OF_SAFETY = 2    # hệ số an toàn chống lật tối thiểu

# Số khoảng chia dùng cho Trapezoidal Rule (phục vụ phần Calculus)
N_SLICES = 1000


# ============================================================
# 1. HÀM TÍNH LỰC NƯỚC BẰNG TÍCH PHÂN (CALCULUS)
# ============================================================

def force_integrand(y, rho=RHO_WATER, g=G):
    """
    f(y) = dF/dy = áp suất * chiều rộng (1 m)
         = (rho * g * y) * 1
    y: độ sâu tính từ mặt nước (0 → D)
    """
    return rho * g * y  # N/m per 1 m chiều dài đập


def trapezoidal_rule(f, a, b, n):
    """
    Xấp xỉ ∫_a^b f(x) dx bằng quy tắc hình thang với n khoảng.
    Dùng để minh hoạ phần Calculus (so sánh với công thức giải tích).
    """
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    integral = h * (0.5 * fx[0] + np.sum(fx[1:-1]) + 0.5 * fx[-1])
    return integral


def compute_hydrostatic_force(height=D, length=L):
    """
    Tính lực nước tổng cộng tác dụng lên mặt thượng lưu,
    cả bằng tích phân số và công thức giải tích.

    Trả về:
        (F_num_total, F_exact_total, M_overturn_per_m)
    """
    # 1. Lực trên 1 mét chiều dài đập (per unit length)
    F_num_per_m   = trapezoidal_rule(force_integrand, 0.0, height, N_SLICES)
    F_exact_per_m = 0.5 * RHO_WATER * G * height**2

    # 2. Nhân với chiều dài đập để ra lực tổng
    F_num_total   = F_num_per_m   * length
    F_exact_total = F_exact_per_m * length

    # 3. Moment gây lật per 1 m chiều dài (dùng cho ổn định)
    #   – lực tổng per 1 m đặt tại h/3 tính từ đáy
    lever_arm = height / 3.0
    M_overturn_per_m = F_exact_per_m * lever_arm

    return F_num_total, F_exact_total, M_overturn_per_m


# ============================================================
# 2. HÀM TÍNH ỔN ĐỊNH CHỐNG LẬT CHO 1 TIẾT DIỆN
# ============================================================

def calculate_dam_stability(dam_height,
                            dam_top_width,
                            dam_base_width,
                            unit_weight_concrete=UNIT_WEIGHT_CONCRETE,
                            rho_water=RHO_WATER,
                            g=G):
    """
    Tính:
        - Moment gây lật do nước (per 1 m chiều dài đập)
        - Moment chống lật do trọng lượng đập
        - Hệ số an toàn chống lật FS = M_resist / M_overturn

    Giả thiết:
        - Mặt thượng lưu thẳng đứng (ở x = 0, từ đáy tới đỉnh)
        - Đáy rộng dam_base_width, đỉnh rộng dam_top_width
        - Mặt hạ lưu nghiêng (như hình minh hoạ)
        - Trục moment lấy tại toe (mép dưới hạ lưu, x = dam_base_width)
    """

    h = dam_height
    T = dam_top_width
    B = dam_base_width

    # -------------------------
    # 2.1. Lực nước & moment lật
    # -------------------------
    F_water_per_m = 0.5 * rho_water * g * h**2          # N/m
    lever_arm_overturn = h / 3.0                        # m – từ toe theo phương đứng
    M_overturn_per_m = F_water_per_m * lever_arm_overturn  # N·m/m

    # -------------------------
    # 2.2. Diện tích và trọng lượng đập
    # -------------------------
    # Diện tích hình thang (tiết diện theo phương đứng)
    # A = (T + B)/2 * h
    A_trap = 0.5 * (T + B) * h      # m^2 per 1 m chiều dài
    W_per_m = A_trap * unit_weight_concrete  # N/m

    # -------------------------
    # 2.3. Tính trọng tâm theo phương ngang
    # -------------------------
    # Cách 1 (đang dùng): chia thành rectangle + triangle, dùng trọng tâm từng phần

    # Phần hình chữ nhật (ngay sát mặt nước, bề rộng = T)
    A_rect = T * h
    x_rect = T / 2.0               # khoảng cách từ heel (x = 0)

    # Phần tam giác (phần chênh từ T → B, nằm phía hạ lưu)
    B_tri = B - T                  # đáy tam giác
    A_tri = 0.5 * B_tri * h
    # Tam giác vuông có góc vuông tại (x=T, y=0),
    # trọng tâm cách góc vuông (b/3, h/3) → x = T + B_tri/3
    x_tri = T + B_tri / 3.0

    # Trọng tâm tổng của hình thang từ heel
    if A_trap == 0:
        raise ValueError("Diện tích hình thang bằng 0 – kiểm tra lại T và B.")
    x_c_from_heel = (A_rect * x_rect + A_tri * x_tri) / A_trap

    # Khoảng cách từ toe (x = B) đến trọng tâm:
    lever_arm_resist = B - x_c_from_heel   # m

    # Moment chống lật per 1 m chiều dài:
    M_resist_per_m = W_per_m * lever_arm_resist  # N·m/m

    # -------------------------
    # 2.4. Hệ số an toàn
    # -------------------------
    FS = M_resist_per_m / M_overturn_per_m

    return M_overturn_per_m, M_resist_per_m, FS


# ============================================================
# 3. THIẾT KẾ BASE WIDTH BẰNG QUÁ TRÌNH LẶP
# ============================================================

def design_dam_base_width(dam_height,
                          dam_top_width,
                          target_FS,
                          unit_weight_concrete=UNIT_WEIGHT_CONCRETE,
                          rho_water=RHO_WATER,
                          g=G,
                          step=0.01,
                          max_base_width=100.0):
    """
    Tìm B_min (dam_base_width) nhỏ nhất sao cho:
        FS(B_min) ≥ target_FS

    Ý tưởng:
        - Bắt đầu từ B = max(T, 0.1)
        - Nếu tại B = T mà FS đã ≥ target → chọn luôn B = T
        - Nếu chưa đủ → tăng B từng bước 'step' cho tới khi FS ≥ target
        - Sau đó refine bằng tìm kiếm nhị phân trong [B - step, B]
          để có B_min mượt hơn.
    """

    T = dam_top_width
    h = dam_height

    def fs_for_B(B):
        _, _, fs_val = calculate_dam_stability(h, T, B,
                                               unit_weight_concrete,
                                               rho_water,
                                               g)
        return fs_val

    # 1. Thử với B = T (hình chữ nhật)
    B0 = max(T, 0.1)
    FS0 = fs_for_B(B0)
    if FS0 >= target_FS:
        return B0

    # 2. Tăng dần B cho tới khi đủ an toàn
    B = B0
    while B <= max_base_width:
        B += step
        FS = fs_for_B(B)
        if FS >= target_FS:
            break
    else:
        # Không tìm được trong khoảng cho phép
        print("Warning: Base width vượt quá giới hạn, thiết kế không thực tế.")
        return B

    # 3. Refine bằng binary search trong [B - step, B]
    low = B - step
    high = B
    for _ in range(40):  # đủ để hội tụ ~10^-6
        mid = 0.5 * (low + high)
        if fs_for_B(mid) >= target_FS:
            high = mid   # vẫn an toàn → có thể thu hẹp xuống
        else:
            low = mid    # không an toàn → phải tăng lên
    return high  # high là B_min thoả FS ≥ target_FS


# ============================================================
# 4. CHẠY TÍNH TOÁN VÀ IN KẾT QUẢ
# ============================================================

if __name__ == "__main__":
    # 4.1. Lực nước và kiểm tra phần Calculus
    F_num_total, F_exact_total, M_overturn_per_m = compute_hydrostatic_force(D, L)

    print("--- Hydrostatic Force on Upstream Face ---")
    print(f"Height D           = {D:.2f} m")
    print(f"Crest Length L     = {L:.2f} m")
    print(f"Numeric Force (Trapezoid Rule) = {F_num_total:,.2f} N")
    print(f"Exact Force (Analytical)       = {F_exact_total:,.2f} N")
    print(f"Difference                    = {abs(F_num_total - F_exact_total):.2e} N\n")

    # 4.2. Thiết kế base width
    designed_base_width = design_dam_base_width(
        dam_height=D,
        dam_top_width=TOP_WIDTH,
        target_FS=TARGET_FACTOR_OF_SAFETY
    )

    # 4.3. Tính lại ổn định với base width thiết kế
    M_ovt, M_res, FS_achieved = calculate_dam_stability(
        dam_height=D,
        dam_top_width=TOP_WIDTH,
        dam_base_width=designed_base_width
    )

    # 4.4. Thể tích bê tông
    area_trap = 0.5 * (TOP_WIDTH + designed_base_width) * D
    total_concrete_volume = area_trap * L  # m^3

    # 4.5. In báo cáo
    print("--- Designed Dam Dimensions and Stability ---")
    print("Fixed Input Parameters:")
    print(f"  Dam Height (D)              = {D:.2f} m")
    print(f"  Crest Length (L)           = {L:.2f} m")
    print(f"  Crest Width (Top Width)    = {TOP_WIDTH:.2f} m")
    print(f"  Unit Weight of Concrete    = {UNIT_WEIGHT_CONCRETE:.2f} N/m^3")
    print(f"  Target FS against overturn = {TARGET_FACTOR_OF_SAFETY:.2f}\n")

    print("Calculated Design Parameters:")
    print(f"  Optimal Base Width (B)     = {designed_base_width:.3f} m")
    print(f"  Trapezoid Area (per m)     = {area_trap:.3f} m^2")
    print(f"  Total Concrete Volume      = {total_concrete_volume:.2f} m^3\n")

    print("Stability Check (per 1 m length of dam):")
    print(f"  Overturning Moment M_ovt   = {M_ovt:,.2f} N·m/m")
    print(f"  Resisting Moment M_res     = {M_res:,.2f} N·m/m")
    print(f"  Achieved Factor of Safety  = {FS_achieved:.3f}")

    if FS_achieved >= TARGET_FACTOR_OF_SAFETY:
        print("  → Dam is STABLE against overturning (FS ≥ target).")
    else:
        print("  → Dam is NOT stable against overturning (FS < target).")


--- Hydrostatic Force on Upstream Face ---
Height D           = 50.00 m
Crest Length L     = 50.00 m
Numeric Force (Trapezoid Rule) = 613,125,000.00 N
Exact Force (Analytical)       = 613,125,000.00 N
Difference                    = 0.00e+00 N

--- Designed Dam Dimensions and Stability ---
Fixed Input Parameters:
  Dam Height (D)              = 50.00 m
  Crest Length (L)           = 50.00 m
  Crest Width (Top Width)    = 2.00 m
  Unit Weight of Concrete    = 24000.00 N/m^3
  Target FS against overturn = 2.00

Calculated Design Parameters:
  Optimal Base Width (B)     = 31.014 m
  Trapezoid Area (per m)     = 825.342 m^2
  Total Concrete Volume      = 41267.09 m^3

Stability Check (per 1 m length of dam):
  Overturning Moment M_ovt   = 204,375,000.00 N·m/m
  Resisting Moment M_res     = 408,750,000.00 N·m/m
  Achieved Factor of Safety  = 2.000
  → Dam is STABLE against overturning (FS ≥ target).
